In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/home/jackdaniel/Documents/learnig skill/tenserflow practice/pima-indians-diabetes.csv")

In [3]:
df.head()

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,6,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,1,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,8,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,1,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


## Normalize the Data

In [ ]:
df.columns

In [ ]:
nor_cols=['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']

In [ ]:
df[nor_cols]=df[nor_cols].apply(lambda x:(x-x.min())/(x.max()-x.min()))

In [ ]:
df.head()

## Feature columns

In [ ]:
import tensorflow as tf

In [ ]:
np=tf.feature_column.numeric_column('Number_pregnant')
gc=tf.feature_column.numeric_column('Glucose_concentration')
bp=tf.feature_column.numeric_column('Blood_pressure')
t=tf.feature_column.numeric_column('Triceps')
i=tf.feature_column.numeric_column('Insulin')
bmi=tf.feature_column.numeric_column('BMI')
p=tf.feature_column.numeric_column('Pedigree')
age=tf.feature_column.numeric_column('Age')

In [ ]:
assigned_group = tf.feature_column.categorical_column_with_vocabulary_list('Group',['A','B','C','D'])

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df["Age"].hist()

In [ ]:
age_bucket=tf.feature_column.bucketized_column(age,boundaries=[20,30,40,50,60,70,80])

In [ ]:
feature_cols=[np,gc,bp,t,i,bmi,p,assigned_group,age_bucket]

## Train test split

In [ ]:
x=df.drop(labels="Class",axis=1)
y=df["Class"]

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=101)

## Estimator

In [ ]:
estimator=tf.estimator.LinearClassifier(feature_columns=feature_cols,n_classes=2)

## Input functions

In [ ]:
#input functions`
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

## Train the model

In [ ]:
#train the model

estimator.train(input_func,steps=1000)

## Evaluation

In [57]:
test_input=tf.estimator.inputs.pandas_input_fn(x=x_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [58]:
result=estimator.evaluate(test_input)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-11T10:25:32Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpo1h9bho9/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-05-11-10:25:34
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.74025977, accuracy_baseline = 0.64935064, auc = 0.8002469, auc_precision_recall = 0.6434016, average_loss = 0.5258714, global_step = 1000, label/mean = 0.35064936, loss = 5.0615125, precision = 0.6666667, prediction/mean = 0.35329816, recall = 0.5185185
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpo1h9bho9/model.ckpt-1000


In [59]:
result

{'accuracy': 0.74025977,
 'accuracy_baseline': 0.64935064,
 'auc': 0.8002469,
 'auc_precision_recall': 0.6434016,
 'average_loss': 0.5258714,
 'label/mean': 0.35064936,
 'loss': 5.0615125,
 'precision': 0.6666667,
 'prediction/mean': 0.35329816,
 'recall': 0.5185185,
 'global_step': 1000}

## predictions

In [88]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(
      x=x_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [89]:
# Predictions is a generator! 
predictions = estimator.predict(pred_input_func)

In [90]:
list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpo1h9bho9/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'logits': array([0.03285873], dtype=float32),
  'logistic': array([0.50821394], dtype=float32),
  'probabilities': array([0.49178606, 0.50821394], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object)},
 {'logits': array([0.46556842], dtype=float32),
  'logistic': array([0.61433434], dtype=float32),
  'probabilities': array([0.38566568, 0.61433434], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object)},
 {'logits': array([-0.60204875], dtype=float32),
  'logistic': array([0.3538751], dtype=float32),
  'probabilities': array([0.64612484, 0.3538751 ], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'all_class_ids': array([0, 1], dtype=int32),
  'all_classes': array([b'0', b'1'], dtype=object)}

## DNN Classifier

In [93]:
model=tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feature_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptuf_kyg6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3a96567550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [94]:
model.train(input_func,steps=1000)

INFO:tensorflow:Calling model_fn.


ValueError: Items of feature_columns must be a <class 'tensorflow.python.feature_column.feature_column_v2.DenseColumn'>. You can wrap a categorical column with an embedding_column or indicator_column. Given: VocabularyListCategoricalColumn(key='Group', vocabulary_list=('A', 'B', 'C', 'D'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

In [98]:
embedded_group=tf.feature_column.embedding_column(assigned_group,  dimension=4)

In [99]:
fet_cols=[np,gc,bp,t,i,bmi,p,embedded_group,age_bucket]

## Estimator

In [100]:
dnn_model=tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=fet_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9fsrej1j', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3a96571400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train the model

In [102]:
dnn_model.train(input_func,steps=1000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp9fsrej1j/model.ckpt.
INFO:tensorflow:loss = 7.97394, step = 1
INFO:tensorflow:global_step/sec: 102.578
INFO:tensorflow:loss = 6.039878, step = 101 (0.980 sec)
INFO:tensorflow:global_step/sec: 120.582
INFO:tensorflow:loss = 6.0490937, step = 201 (0.838 sec)
INFO:tensorflow:global_step/sec: 159.971
INFO:tensorflow:loss = 6.4114103, step = 301 (0.626

INFO:tensorflow:global_step/sec: 142.154
INFO:tensorflow:loss = 4.8426504, step = 601 (0.708 sec)
INFO:tensorflow:global_step/sec: 135.01
INFO:tensorflow:loss = 4.155343, step = 701 (0.728 sec)
INFO:tensorflow:global_step/sec: 125.996
INFO:tensorflow:loss = 3.072501, step = 801 (0.807 sec)
INFO:tensorflow:global_step/sec: 129.097
INFO:tensorflow:loss = 4.718802, step = 901 (0.761 sec)
INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmp9fsrej1j/model.ckpt.
INFO:tensorflow:Loss for final step: 5.0501924.


## Evaluation

In [103]:
dnn_model.evaluate(test_input)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-05-11T11:31:24Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9fsrej1j/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-05-11-11:31:26
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.77056277, accuracy_baseline = 0.64935064, auc = 0.8346914, auc_precision_recall = 0.7042823, average_loss = 0.478284, global_step = 1000, label/mean = 0.35064936, loss = 4.6034837, precision = 0.68421054, prediction/mean = 0.33653966, recall = 0.6419753
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmp9fsrej1j/model.ckpt-1000


{'accuracy': 0.77056277,
 'accuracy_baseline': 0.64935064,
 'auc': 0.8346914,
 'auc_precision_recall': 0.7042823,
 'average_loss': 0.478284,
 'label/mean': 0.35064936,
 'loss': 4.6034837,
 'precision': 0.68421054,
 'prediction/mean': 0.33653966,
 'recall': 0.6419753,
 'global_step': 1000}